In [ ]:
# notebooks/05_backtesting_strategies.ipynb
# Run in: VS Code or Colab

{
 "cells": [
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "# Backtesting Trading Strategies\n",
    "Test and evaluate different trading strategies on historical data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "import sys\n",
    "from pathlib import Path\n",
    "import pandas as pd\n",
    "import numpy as np\n",
    "import matplotlib.pyplot as plt\n",
    "import seaborn as sns\n",
    "import warnings\n",
    "warnings.filterwarnings('ignore')\n",
    "\n",
    "project_root = Path.cwd().parent\n",
    "sys.path.insert(0, str(project_root))\n",
    "\n",
    "from src.data.data_loader import DataLoader\n",
    "from src.data.technical_indicators import TechnicalIndicators\n",
    "from src.portfolio.backtesting_engine import BacktestingEngine\n",
    "from src.trading_signals.strategy_engine import StrategyEngine\n",
    "\n",
    "plt.style.use('seaborn-v0_8-darkgrid')\n",
    "%matplotlib inline"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 1. Load Historical Data"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "data_loader = DataLoader()\n",
    "tech_indicators = TechnicalIndicators()\n",
    "backtesting_engine = BacktestingEngine()\n",
    "strategy_engine = StrategyEngine()\n",
    "\n",
    "symbol = 'AAPL'\n",
    "df = data_loader.load_stock_data(symbol, period='3y')\n",
    "df_with_indicators = tech_indicators.add_all_indicators(df)\n",
    "\n",
    "print(f\"Data loaded: {len(df)} days\")\n",
    "print(f\"Date range: {df.index.min()} to {df.index.max()}\")\n",
    "\n",
    "plt.figure(figsize=(14, 6))\n",
    "plt.plot(df.index, df['Close'], linewidth=2)\n",
    "plt.title(f'{symbol} Historical Price', fontweight='bold', fontsize=14)\n",
    "plt.xlabel('Date')\n",
    "plt.ylabel('Price ($)')\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 2. Buy and Hold Strategy (Baseline)"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "initial_capital = 10000\n",
    "\n",
    "buy_price = df['Close'].iloc[0]\n",
    "sell_price = df['Close'].iloc[-1]\n",
    "shares = initial_capital / buy_price\n",
    "final_value = shares * sell_price\n",
    "buy_hold_return = ((final_value - initial_capital) / initial_capital) * 100\n",
    "\n",
    "print(f\"\\nBuy and Hold Strategy:\")\n",
    "print(f\"Initial Investment: ${initial_capital:,.2f}\")\n",
    "print(f\"Buy Price: ${buy_price:.2f}\")\n",
    "print(f\"Sell Price: ${sell_price:.2f}\")\n",
    "print(f\"Final Value: ${final_value:,.2f}\")\n",
    "print(f\"Total Return: {buy_hold_return:.2f}%\")\n",
    "\n",
    "buy_hold_equity = []\n",
    "for price in df['Close']:\n",
    "    buy_hold_equity.append(shares * price)\n",
    "\n",
    "plt.figure(figsize=(14, 6))\n",
    "plt.plot(df.index, buy_hold_equity, linewidth=2, label='Buy & Hold')\n",
    "plt.axhline(y=initial_capital, color='r', linestyle='--', label='Initial Capital')\n",
    "plt.title('Buy and Hold Portfolio Value', fontweight='bold', fontsize=14)\n",
    "plt.xlabel('Date')\n",
    "plt.ylabel('Portfolio Value ($)')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 3. Moving Average Crossover Strategy"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"\\n=== Moving Average Crossover Strategy ===\")\n",
    "\n",
    "strategy_params = {\n",
    "    'type': 'ma_crossover',\n",
    "    'short_window': 20,\n",
    "    'long_window': 50\n",
    "}\n",
    "\n",
    "backtest_config = {\n",
    "    'initial_capital': initial_capital,\n",
    "    'position_size': 1.0,\n",
    "    'commission': 1.0\n",
    "}\n",
    "\n",
    "ma_results = backtesting_engine.run_backtest(df_with_indicators, strategy_params, backtest_config)\n",
    "\n",
    "print(f\"\\nMoving Average Crossover Results:\")\n",
    "print(f\"Final Portfolio Value: ${ma_results['final_value']:,.2f}\")\n",
    "print(f\"Total Return: {((ma_results['final_value'] - initial_capital) / initial_capital * 100):.2f}%\")\n",
    "print(f\"Total Trades: {ma_results['total_trades']}\")\n",
    "print(f\"Winning Trades: {ma_results['winning_trades']}\")\n",
    "print(f\"Losing Trades: {ma_results['losing_trades']}\")\n",
    "print(f\"Win Rate: {(ma_results['winning_trades'] / ma_results['total_trades'] * 100) if ma_results['total_trades'] > 0 else 0:.2f}%\")\n",
    "print(f\"Sharpe Ratio: {ma_results['sharpe_ratio']:.2f}\")\n",
    "print(f\"Max Drawdown: {ma_results['max_drawdown']:.2f}%\")\n",
    "\n",
    "plt.figure(figsize=(14, 8))\n",
    "\n",
    "plt.subplot(2, 1, 1)\n",
    "plt.plot(df.index, df['Close'], label='Price', linewidth=2)\n",
    "plt.plot(df.index, df_with_indicators['SMA_20'], label='SMA 20', linestyle='--', alpha=0.7)\n",
    "plt.plot(df.index, df_with_indicators['SMA_50'], label='SMA 50', linestyle='--', alpha=0.7)\n",
    "\n",
    "trades_df = pd.DataFrame(ma_results['trades'])\n",
    "if not trades_df.empty:\n",
    "    buy_signals = trades_df[trades_df['type'] == 'BUY']\n",
    "    sell_signals = trades_df[trades_df['type'] == 'SELL']\n",
    "    \n",
    "    for _, trade in buy_signals.iterrows():\n",
    "        plt.scatter(trade['date'], trade['price'], color='green', marker='^', s=100, zorder=5)\n",
    "    for _, trade in sell_signals.iterrows():\n",
    "        plt.scatter(trade['date'], trade['price'], color='red', marker='v', s=100, zorder=5)\n",
    "\n",
    "plt.title('Moving Average Crossover Signals', fontweight='bold')\n",
    "plt.ylabel('Price ($)')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "plt.subplot(2, 1, 2)\n",
    "portfolio_history = pd.DataFrame(ma_results['portfolio_history'])\n",
    "plt.plot(portfolio_history['Date'], portfolio_history['Portfolio Value'], linewidth=2, label='MA Strategy')\n",
    "plt.plot(df.index, buy_hold_equity, linewidth=2, alpha=0.7, label='Buy & Hold')\n",
    "plt.axhline(y=initial_capital, color='r', linestyle='--', alpha=0.5, label='Initial Capital')\n",
    "plt.title('Portfolio Value Comparison', fontweight='bold')\n",
    "plt.xlabel('Date')\n",
    "plt.ylabel('Portfolio Value ($)')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 4. RSI Strategy"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"\\n=== RSI Strategy ===\")\n",
    "\n",
    "rsi_strategy_params = {\n",
    "    'type': 'rsi',\n",
    "    'period': 14,\n",
    "    'oversold': 30,\n",
    "    'overbought': 70\n",
    "}\n",
    "\n",
    "rsi_results = backtesting_engine.run_backtest(df_with_indicators, rsi_strategy_params, backtest_config)\n",
    "\n",
    "print(f\"\\nRSI Strategy Results:\")\n",
    "print(f\"Final Portfolio Value: ${rsi_results['final_value']:,.2f}\")\n",
    "print(f\"Total Return: {((rsi_results['final_value'] - initial_capital) / initial_capital * 100):.2f}%\")\n",
    "print(f\"Total Trades: {rsi_results['total_trades']}\")\n",
    "print(f\"Winning Trades: {rsi_results['winning_trades']}\")\n",
    "print(f\"Losing Trades: {rsi_results['losing_trades']}\")\n",
    "print(f\"Win Rate: {(rsi_results['winning_trades'] / rsi_results['total_trades'] * 100) if rsi_results['total_trades'] > 0 else 0:.2f}%\")\n",
    "print(f\"Sharpe Ratio: {rsi_results['sharpe_ratio']:.2f}\")\n",
    "print(f\"Max Drawdown: {rsi_results['max_drawdown']:.2f}%\")\n",
    "\n",
    "plt.figure(figsize=(14, 10))\n",
    "\n",
    "plt.subplot(3, 1, 1)\n",
    "plt.plot(df.index, df['Close'], label='Price', linewidth=2)\n",
    "rsi_trades_df = pd.DataFrame(rsi_results['trades'])\n",
    "if not rsi_trades_df.empty:\n",
    "    buy_signals = rsi_trades_df[rsi_trades_df['type'] == 'BUY']\n",
    "    sell_signals = rsi_trades_df[rsi_trades_df['type'] == 'SELL']\n",
    "    \n",
    "    for _, trade in buy_signals.iterrows():\n",
    "        plt.scatter(trade['date'], trade['price'], color='green', marker='^', s=100, zorder=5)\n",
    "    for _, trade in sell_signals.iterrows():\n",
    "        plt.scatter(trade['date'], trade['price'], color='red', marker='v', s=100, zorder=5)\n",
    "\n",
    "plt.title('RSI Strategy Signals', fontweight='bold')\n",
    "plt.ylabel('Price ($)')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "plt.subplot(3, 1, 2)\n",
    "plt.plot(df.index, df_with_indicators['RSI'], linewidth=2, color='purple')\n",
    "plt.axhline(y=70, color='r', linestyle='--', label='Overbought')\n",
    "plt.axhline(y=30, color='g', linestyle='--', label='Oversold')\n",
    "plt.title('RSI Indicator', fontweight='bold')\n",
    "plt.ylabel('RSI')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "plt.subplot(3, 1, 3)\n",
    "rsi_portfolio_history = pd.DataFrame(rsi_results['portfolio_history'])\n",
    "plt.plot(rsi_portfolio_history['Date'], rsi_portfolio_history['Portfolio Value'], linewidth=2, label='RSI Strategy')\n",
    "plt.plot(df.index, buy_hold_equity, linewidth=2, alpha=0.7, label='Buy & Hold')\n",
    "plt.axhline(y=initial_capital, color='r', linestyle='--', alpha=0.5, label='Initial Capital')\n",
    "plt.title('Portfolio Value Comparison', fontweight='bold')\n",
    "plt.xlabel('Date')\n",
    "plt.ylabel('Portfolio Value ($)')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 5. MACD Strategy"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"\\n=== MACD Strategy ===\")\n",
    "\n",
    "macd_strategy_params = {\n",
    "    'type': 'macd',\n",
    "    'fast_period': 12,\n",
    "    'slow_period': 26,\n",
    "    'signal_period': 9\n",
    "}\n",
    "\n",
    "macd_results = backtesting_engine.run_backtest(df_with_indicators, macd_strategy_params, backtest_config)\n",
    "\n",
    "print(f\"\\nMACD Strategy Results:\")\n",
    "print(f\"Final Portfolio Value: ${macd_results['final_value']:,.2f}\")\n",
    "print(f\"Total Return: {((macd_results['final_value'] - initial_capital) / initial_capital * 100):.2f}%\")\n",
    "print(f\"Total Trades: {macd_results['total_trades']}\")\n",
    "print(f\"Winning Trades: {macd_results['winning_trades']}\")\n",
    "print(f\"Losing Trades: {macd_results['losing_trades']}\")\n",
    "print(f\"Win Rate: {(macd_results['winning_trades'] / macd_results['total_trades'] * 100) if macd_results['total_trades'] > 0 else 0:.2f}%\")\n",
    "print(f\"Sharpe Ratio: {macd_results['sharpe_ratio']:.2f}\")\n",
    "print(f\"Max Drawdown: {macd_results['max_drawdown']:.2f}%\")\n",
    "\n",
    "plt.figure(figsize=(14, 10))\n",
    "\n",
    "plt.subplot(3, 1, 1)\n",
    "plt.plot(df.index, df['Close'], label='Price', linewidth=2)\n",
    "macd_trades_df = pd.DataFrame(macd_results['trades'])\n",
    "if not macd_trades_df.empty:\n",
    "    buy_signals = macd_trades_df[macd_trades_df['type'] == 'BUY']\n",
    "    sell_signals = macd_trades_df[macd_trades_df['type'] == 'SELL']\n",
    "    \n",
    "    for _, trade in buy_signals.iterrows():\n",
    "        plt.scatter(trade['date'], trade['price'], color='green', marker='^', s=100, zorder=5)\n",
    "    for _, trade in sell_signals.iterrows():\n",
    "        plt.scatter(trade['date'], trade['price'], color='red', marker='v', s=100, zorder=5)\n",
    "\n",
    "plt.title('MACD Strategy Signals', fontweight='bold')\n",
    "plt.ylabel('Price ($)')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "plt.subplot(3, 1, 2)\n",
    "plt.plot(df.index, df_with_indicators['MACD'], label='MACD', linewidth=2)\n",
    "plt.plot(df.index, df_with_indicators['MACD_Signal'], label='Signal', linewidth=2)\n",
    "plt.bar(df.index, df_with_indicators['MACD_Histogram'], label='Histogram', alpha=0.3)\n",
    "plt.title('MACD Indicator', fontweight='bold')\n",
    "plt.ylabel('MACD')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "plt.subplot(3, 1, 3)\n",
    "macd_portfolio_history = pd.DataFrame(macd_results['portfolio_history'])\n",
    "plt.plot(macd_portfolio_history['Date'], macd_portfolio_history['Portfolio Value'], linewidth=2, label='MACD Strategy')\n",
    "plt.plot(df.index, buy_hold_equity, linewidth=2, alpha=0.7, label='Buy & Hold')\n",
    "plt.axhline(y=initial_capital, color='r', linestyle='--', alpha=0.5, label='Initial Capital')\n",
    "plt.title('Portfolio Value Comparison', fontweight='bold')\n",
    "plt.xlabel('Date')\n",
    "plt.ylabel('Portfolio Value ($)')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 6. Bollinger Bands Strategy"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "print(\"\\n=== Bollinger Bands Strategy ===\")\n",
    "\n",
    "bb_strategy_params = {\n",
    "    'type': 'bollinger',\n",
    "    'period': 20,\n",
    "    'std_dev': 2.0\n",
    "}\n",
    "\n",
    "bb_results = backtesting_engine.run_backtest(df_with_indicators, bb_strategy_params, backtest_config)\n",
    "\n",
    "print(f\"\\nBollinger Bands Strategy Results:\")\n",
    "print(f\"Final Portfolio Value: ${bb_results['final_value']:,.2f}\")\n",
    "print(f\"Total Return: {((bb_results['final_value'] - initial_capital) / initial_capital * 100):.2f}%\")\n",
    "print(f\"Total Trades: {bb_results['total_trades']}\")\n",
    "print(f\"Winning Trades: {bb_results['winning_trades']}\")\n",
    "print(f\"Losing Trades: {bb_results['losing_trades']}\")\n",
    "print(f\"Win Rate: {(bb_results['winning_trades'] / bb_results['total_trades'] * 100) if bb_results['total_trades'] > 0 else 0:.2f}%\")\n",
    "print(f\"Sharpe Ratio: {bb_results['sharpe_ratio']:.2f}\")\n",
    "print(f\"Max Drawdown: {bb_results['max_drawdown']:.2f}%\")\n",
    "\n",
    "plt.figure(figsize=(14, 8))\n",
    "\n",
    "plt.subplot(2, 1, 1)\n",
    "plt.plot(df.index, df['Close'], label='Price', linewidth=2, color='black')\n",
    "plt.plot(df.index, df_with_indicators['BB_Upper'], label='Upper Band', linestyle='--', alpha=0.7)\n",
    "plt.plot(df.index, df_with_indicators['BB_Middle'], label='Middle Band', linestyle='--', alpha=0.7)\n",
    "plt.plot(df.index, df_with_indicators['BB_Lower'], label='Lower Band', linestyle='--', alpha=0.7)\n",
    "plt.fill_between(df.index, df_with_indicators['BB_Upper'], df_with_indicators['BB_Lower'], alpha=0.1)\n",
    "\n",
    "bb_trades_df = pd.DataFrame(bb_results['trades'])\n",
    "if not bb_trades_df.empty:\n",
    "    buy_signals = bb_trades_df[bb_trades_df['type'] == 'BUY']\n",
    "    sell_signals = bb_trades_df[bb_trades_df['type'] == 'SELL']\n",
    "    \n",
    "    for _, trade in buy_signals.iterrows():\n",
    "        plt.scatter(trade['date'], trade['price'], color='green', marker='^', s=100, zorder=5)\n",
    "    for _, trade in sell_signals.iterrows():\n",
    "        plt.scatter(trade['date'], trade['price'], color='red', marker='v', s=100, zorder=5)\n",
    "\n",
    "plt.title('Bollinger Bands Strategy Signals', fontweight='bold')\n",
    "plt.ylabel('Price ($)')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "plt.subplot(2, 1, 2)\n",
    "bb_portfolio_history = pd.DataFrame(bb_results['portfolio_history'])\n",
    "plt.plot(bb_portfolio_history['Date'], bb_portfolio_history['Portfolio Value'], linewidth=2, label='BB Strategy')\n",
    "plt.plot(df.index, buy_hold_equity, linewidth=2, alpha=0.7, label='Buy & Hold')\n",
    "plt.axhline(y=initial_capital, color='r', linestyle='--', alpha=0.5, label='Initial Capital')\n",
    "plt.title('Portfolio Value Comparison', fontweight='bold')\n",
    "plt.xlabel('Date')\n",
    "plt.ylabel('Portfolio Value ($)')\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 7. Strategy Comparison"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "comparison_data = [\n",
    "    {\n",
    "        'Strategy': 'Buy & Hold',\n",
    "        'Final Value': final_value,\n",
    "        'Total Return (%)': buy_hold_return,\n",
    "        'Total Trades': 1,\n",
    "        'Win Rate (%)': 100 if buy_hold_return > 0 else 0,\n",
    "        'Sharpe Ratio': 0,\n",
    "        'Max Drawdown (%)': 0\n",
    "    },\n",
    "    {\n",
    "        'Strategy': 'MA Crossover',\n",
    "        'Final Value': ma_results['final_value'],\n",
    "        'Total Return (%)': ((ma_results['final_value'] - initial_capital) / initial_capital * 100),\n",
    "        'Total Trades': ma_results['total_trades'],\n",
    "        'Win Rate (%)': (ma_results['winning_trades'] / ma_results['total_trades'] * 100) if ma_results['total_trades'] > 0 else 0,\n",
    "        'Sharpe Ratio': ma_results['sharpe_ratio'],\n",
    "        'Max Drawdown (%)': ma_results['max_drawdown']\n",
    "    },\n",
    "    {\n",
    "        'Strategy': 'RSI',\n",
    "        'Final Value': rsi_results['final_value'],\n",
    "        'Total Return (%)': ((rsi_results['final_value'] - initial_capital) / initial_capital * 100),\n",
    "        'Total Trades': rsi_results['total_trades'],\n",
    "        'Win Rate (%)': (rsi_results['winning_trades'] / rsi_results['total_trades'] * 100) if rsi_results['total_trades'] > 0 else 0,\n",
    "        'Sharpe Ratio': rsi_results['sharpe_ratio'],\n",
    "        'Max Drawdown (%)': rsi_results['max_drawdown']\n",
    "    },\n",
    "    {\n",
    "        'Strategy': 'MACD',\n",
    "        'Final Value': macd_results['final_value'],\n",
    "        'Total Return (%)': ((macd_results['final_value'] - initial_capital) / initial_capital * 100),\n",
    "        'Total Trades': macd_results['total_trades'],\n",
    "        'Win Rate (%)': (macd_results['winning_trades'] / macd_results['total_trades'] * 100) if macd_results['total_trades'] > 0 else 0,\n",
    "        'Sharpe Ratio': macd_results['sharpe_ratio'],\n",
    "        'Max Drawdown (%)': macd_results['max_drawdown']\n",
    "    },\n",
    "    {\n",
    "        'Strategy': 'Bollinger Bands',\n",
    "        'Final Value': bb_results['final_value'],\n",
    "        'Total Return (%)': ((bb_results['final_value'] - initial_capital) / initial_capital * 100),\n",
    "        'Total Trades': bb_results['total_trades'],\n",
    "        'Win Rate (%)': (bb_results['winning_trades'] / bb_results['total_trades'] * 100) if bb_results['total_trades'] > 0 else 0,\n",
    "        'Sharpe Ratio': bb_results['sharpe_ratio'],\n",
    "        'Max Drawdown (%)': bb_results['max_drawdown']\n",
    "    }\n",
    "]\n",
    "\n",
    "comparison_df = pd.DataFrame(comparison_data)\n",
    "\n",
    "print(\"\\n\" + \"=\"*80)\n",
    "print(\"STRATEGY COMPARISON\")\n",
    "print(\"=\"*80)\n",
    "print(comparison_df.to_string(index=False))\n",
    "print(\"=\"*80)\n",
    "\n",
    "fig, axes = plt.subplots(2, 3, figsize=(18, 10))\n",
    "\n",
    "comparison_df.plot(x='Strategy', y='Total Return (%)', kind='bar', ax=axes[0, 0], legend=False, color='steelblue')\n",
    "axes[0, 0].set_title('Total Return', fontweight='bold')\n",
    "axes[0, 0].set_ylabel('Return (%)')\n",
    "axes[0, 0].tick_params(axis='x', rotation=45)\n",
    "axes[0, 0].grid(True, alpha=0.3)\n",
    "\n",
    "comparison_df.plot(x='Strategy', y='Sharpe Ratio', kind='bar', ax=axes[0, 1], legend=False, color='coral')\n",
    "axes[0, 1].set_title('Sharpe Ratio', fontweight='bold')\n",
    "axes[0, 1].set_ylabel('Ratio')\n",
    "axes[0, 1].tick_params(axis='x', rotation=45)\n",
    "axes[0, 1].grid(True, alpha=0.3)\n",
    "\n",
    "comparison_df.plot(x='Strategy', y='Max Drawdown (%)', kind='bar', ax=axes[0, 2], legend=False, color='lightcoral')\n",
    "axes[0, 2].set_title('Max Drawdown', fontweight='bold')\n",
    "axes[0, 2].set_ylabel('Drawdown (%)')\n",
    "axes[0, 2].tick_params(axis='x', rotation=45)\n",
    "axes[0, 2].grid(True, alpha=0.3)\n",
    "\n",
    "comparison_df.plot(x='Strategy', y='Total Trades', kind='bar', ax=axes[1, 0], legend=False, color='lightgreen')\n",
    "axes[1, 0].set_title('Total Trades', fontweight='bold')\n",
    "axes[1, 0].set_ylabel('Number of Trades')\n",
    "axes[1, 0].tick_params(axis='x', rotation=45)\n",
    "axes[1, 0].grid(True, alpha=0.3)\n",
    "\n",
    "comparison_df.plot(x='Strategy', y='Win Rate (%)', kind='bar', ax=axes[1, 1], legend=False, color='gold')\n",
    "axes[1, 1].set_title('Win Rate', fontweight='bold')\n",
    "axes[1, 1].set_ylabel('Win Rate (%)')\n",
    "axes[1, 1].tick_params(axis='x', rotation=45)\n",
    "axes[1, 1].grid(True, alpha=0.3)\n",
    "\n",
    "comparison_df.plot(x='Strategy', y='Final Value', kind='bar', ax=axes[1, 2], legend=False, color='mediumpurple')\n",
    "axes[1, 2].set_title('Final Portfolio Value', fontweight='bold')\n",
    "axes[1, 2].set_ylabel('Value ($)')\n",
    "axes[1, 2].tick_params(axis='x', rotation=45)\n",
    "axes[1, 2].grid(True, alpha=0.3)\n",
    "\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 8. All Strategies Portfolio Growth"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(16, 8))\n",
    "\n",
    "plt.plot(df.index, buy_hold_equity, linewidth=3, label='Buy & Hold', alpha=0.9)\n",
    "plt.plot(portfolio_history['Date'], portfolio_history['Portfolio Value'], linewidth=2, label='MA Crossover', alpha=0.8)\n",
    "plt.plot(rsi_portfolio_history['Date'], rsi_portfolio_history['Portfolio Value'], linewidth=2, label='RSI', alpha=0.8)\n",
    "plt.plot(macd_portfolio_history['Date'], macd_portfolio_history['Portfolio Value'], linewidth=2, label='MACD', alpha=0.8)\n",
    "plt.plot(bb_portfolio_history['Date'], bb_portfolio_history['Portfolio Value'], linewidth=2, label='Bollinger Bands', alpha=0.8)\n",
    "\n",
    "plt.axhline(y=initial_capital, color='red', linestyle='--', linewidth=2, alpha=0.5, label='Initial Capital')\n",
    "\n",
    "plt.title('All Strategies Portfolio Value Comparison', fontweight='bold', fontsize=16)\n",
    "plt.xlabel('Date', fontsize=12)\n",
    "plt.ylabel('Portfolio Value ($)', fontsize=12)\n",
    "plt.legend(fontsize=11)\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 9. Risk-Adjusted Performance"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "plt.figure(figsize=(12, 8))\n",
    "\n",
    "for _, row in comparison_df.iterrows():\n",
    "    if row['Strategy'] != 'Buy & Hold':\n",
    "        plt.scatter(row['Max Drawdown (%)'], row['Total Return (%)'], \n",
    "                   s=row['Sharpe Ratio']*100, alpha=0.6, label=row['Strategy'])\n",
    "\n",
    "plt.xlabel('Max Drawdown (%)', fontsize=12)\n",
    "plt.ylabel('Total Return (%)', fontsize=12)\n",
    "plt.title('Risk vs Return (Bubble size = Sharpe Ratio)', fontweight='bold', fontsize=14)\n",
    "plt.legend()\n",
    "plt.grid(True, alpha=0.3)\n",
    "plt.axhline(y=0, color='black', linestyle='-', linewidth=0.5)\n",
    "plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)\n",
    "plt.tight_layout()\n",
    "plt.show()"
   ]
  },
  {
   "cell_type": "markdown",
   "metadata": {},
   "source": [
    "## 10. Conclusion and Recommendations"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "metadata": {},
   "outputs": [],
   "source": [
    "best_strategy = comparison_df.loc[comparison_df['Total Return (%)'].idxmax()]\n",
    "best_sharpe = comparison_df.loc[comparison_df['Sharpe Ratio'].idxmax()]\n",
    "lowest_drawdown = comparison_df.loc[comparison_df['Max Drawdown (%)'].idxmin()]\n",
    "\n",
    "print(\"\\n\" + \"=\"*80)\n",
    "print(\"BACKTESTING SUMMARY & RECOMMENDATIONS\")\n",
    "print(\"=\"*80)\n",
    "\n",
    "print(f\"\\n1. Best Overall Return:\")\n",
    "print(f\"   Strategy: {best_strategy['Strategy']}\")\n",
    "print(f\"   Total Return: {best_strategy['Total Return (%)']:.2f}%\")\n",
    "print(f\"   Final Value: ${best_strategy['Final Value']:,.2f}\")\n",
    "\n",
    "print(f\"\\n2. Best Risk-Adjusted Return:\")\n",
    "print(f\"   Strategy: {best_sharpe['Strategy']}\")\n",
    "print(f\"   Sharpe Ratio: {best_sharpe['Sharpe Ratio']:.2f}\")\n",
    "print(f\"   Total Return: {best_sharpe['Total Return (%)']:.2f}%\")\n",
    "\n",
    "print(f\"\\n3. Lowest Drawdown:\")\n",
    "print(f\"   Strategy: {lowest_drawdown['Strategy']}\")\n",
    "print(f\"   Max Drawdown: {lowest_drawdown['Max Drawdown (%)']:.2f}%\")\n",
    "\n",
    "print(f\"\\n4. Key Insights:\")\n",
    "avg_return = comparison_df[comparison_df['Strategy'] != 'Buy & Hold']['Total Return (%)'].mean()\n",
    "print(f\"   - Average strategy return: {avg_return:.2f}% vs Buy & Hold: {buy_hold_return:.2f}%\")\n",
    "print(f\"   - Strategies with positive Sharpe ratio indicate better risk-adjusted returns\")\n",
    "print(f\"   - Lower max drawdown means less volatility and risk\")\n",
    "print(f\"   - Higher win rate doesn't always mean better overall returns\")\n",
    "\n",
    "print(f\"\\n5. Recommendations:\")\n",
    "print(f\"   - Consider combining multiple strategies for diversification\")\n",
    "print(f\"   - Monitor and adjust strategies based on market conditions\")\n",
    "print(f\"   - Always use stop-loss orders to limit downside risk\")\n",
    "print(f\"   - Past performance does not guarantee future results\")\n",
    "\n",
    "print(\"\\n\" + \"=\"*80)"
   ]
  }
 ],
 "metadata": {
  "kernelspec": {
   "display_name": "Python 3",
   "language": "python",
   "name": "python3"
  },
  "language_info": {
   "codemirror_mode": {
    "name": "ipython",
    "version": 3
   },
   "file_extension": ".py",
   "mimetype": "text/x-python",
   "name": "python",
   "nbconvert_exporter": "python",
   "pygments_lexer": "ipython3",
   "version": "3.9.0"
  }
 },
 "nbformat": 4,
 "nbformat_minor": 4
}